## The `datetime` Module – Advanced Exercises

This notebook contains a series of advanced (but not too advanced) exercises focusing on the `datetime` module:

- `date`, `time`, `datetime`
- `timedelta`
- `timezone` and aware vs naive datetimes
- Parsing and formatting dates and datetimes

Each exercise has:
- A problem description (markdown)
- A solution (code), written using best practices

You can collapse the solution cells and re-implement them yourself for practice.


---
### Exercise 1 – Business Days Between Two Dates (Using `date` and `timedelta`)

Goal: Compute the number of business days (Monday–Friday) between two dates.

Requirements:

1. Implement a function:
   ```python
   def business_days_between(start: datetime.date, end: datetime.date) -> int:
       ...
   ```
   that returns the number of business days from `start` (inclusive) to `end` (exclusive).
2. If `start >= end`, return `0`.
3. Count only days where `weekday()` is in `0..4` (Monday–Friday).
4. Demonstrate the function with a few example ranges, including:
   - A range within a single week
   - A range that crosses weekends
   - A reversed range (end before start)

This exercise practices:
- Iterating over dates with `timedelta(days=1)`
- Using `date.weekday()`
- Being precise about inclusive and exclusive ranges


In [1]:
from __future__ import annotations
import datetime as _dt


def business_days_between(start: _dt.date, end: _dt.date) -> int:
    '''Return the number of business days between start (inclusive) and end (exclusive).

    Business days are Monday–Friday. If start >= end, returns 0.
    '''
    if start >= end:
        return 0

    count = 0
    current = start
    one_day = _dt.timedelta(days=1)

    while current < end:
        if current.weekday() < 5:  # 0=Monday, 6=Sunday
            count += 1
        current += one_day

    return count


if __name__ == '__main__':
    # Within a single week (Mon–Fri)
    monday = _dt.date(2023, 1, 2)
    friday = _dt.date(2023, 1, 6)
    print('Mon..Fri (exclusive of Fri):', business_days_between(monday, friday))

    # Range that crosses a weekend
    friday = _dt.date(2023, 1, 6)
    next_monday = _dt.date(2023, 1, 9)
    print('Fri..next Mon:', business_days_between(friday, next_monday))

    # Reversed range
    print('Reversed (end before start):', business_days_between(friday, monday))


Mon..Fri (exclusive of Fri): 4
Fri..next Mon: 1
Reversed (end before start): 0


---
### Exercise 2 – Parsing and Validating Date Ranges (`fromisoformat`)

Goal: Parse a compact date range string into two `date` objects and validate it.

Scenario: You receive input like `"2023-01-01:2023-01-31"` and want to convert it into a `(start_date, end_date)` tuple.

Requirements:

1. Implement a function:
   ```python
   def parse_date_range(range_str: str) -> tuple[datetime.date, datetime.date]:
       ...
   ```
2. `range_str` must be in the form `YYYY-MM-DD:YYYY-MM-DD`.
   - Use `datetime.date.fromisoformat` so you get proper validation of the dates.
3. If the string is malformed or dates cannot be parsed, raise `ValueError` with a clear message.
4. If `start > end`, also raise `ValueError` (you do not allow inverted ranges here).
5. Demonstrate the function with:
   - A valid range
   - An invalid date
   - An inverted range
   - A range with an invalid separator

This exercise practices:
- Parsing ISO 8601-like dates
- Validating user input
- Raising appropriate errors for bad data


In [2]:
import datetime as _dt


def parse_date_range(range_str: str) -> tuple[_dt.date, _dt.date]:
    '''Parse a date range of the form 'YYYY-MM-DD:YYYY-MM-DD'.

    Raises ValueError if the format is invalid or start > end.
    '''
    if ':' not in range_str:
        raise ValueError(f"Invalid range format (missing ':'): {range_str!r}")

    start_str, end_str = range_str.split(':', maxsplit=1)

    try:
        start = _dt.date.fromisoformat(start_str)
        end = _dt.date.fromisoformat(end_str)
    except ValueError as exc:
        raise ValueError(f'Invalid date in range {range_str!r}: {exc}') from exc

    if start > end:
        raise ValueError(f'Start date {start} is after end date {end}')

    return start, end


if __name__ == '__main__':
    examples = [
        '2023-01-01:2023-01-31',  # valid
        '2023-02-30:2023-03-01',  # invalid date (Feb 30)
        '2023-03-10:2023-03-01',  # inverted range
        '2023-01-01/2023-01-31',  # wrong separator
    ]

    for s in examples:
        print(f'Parsing {s!r}:')
        try:
            start, end = parse_date_range(s)
            print('  ->', start, 'to', end)
        except ValueError as exc:
            print('  Error:', exc)
        print('-')


Parsing '2023-01-01:2023-01-31':
  -> 2023-01-01 to 2023-01-31
-
Parsing '2023-02-30:2023-03-01':
  Error: Invalid date in range '2023-02-30:2023-03-01': day is out of range for month
-
Parsing '2023-03-10:2023-03-01':
  Error: Start date 2023-03-10 is after end date 2023-03-01
-
Parsing '2023-01-01/2023-01-31':
  Error: Invalid range format (missing ':'): '2023-01-01/2023-01-31'
-


---
### Exercise 3 – Rounding `datetime` Objects to Minute or Hour

Goal: Implement a helper that rounds a `datetime` to the nearest minute or hour.

Requirements:

1. Implement a function:
   ```python
   def round_datetime(dt: datetime.datetime, *, to: str = 'minute') -> datetime.datetime:
       ...
   ```
2. `to` can be:
   - `minute`: round to the nearest minute
   - `hour`: round to the nearest hour
3. Use standard half-up rounding:
   - For minutes: if `seconds >= 30` round up, otherwise down.
   - For hours: if `minutes >= 30` round up, otherwise down (ignore seconds).
4. Do not modify the input `dt` in-place; always return a new `datetime`.
5. Preserve any `tzinfo` on the original `datetime`.
6. Demonstrate the function on various examples, for example:
   - `13:29:40` rounded to minute
   - `13:29:40` rounded to hour
   - `13:31:10` rounded to hour

This exercise practices:
- Working with `datetime` components
- `timedelta` arithmetic for rounding
- Preserving time zone information (`tzinfo`)


In [3]:
import datetime as _dt
from typing import Literal


def round_datetime(
    dt: _dt.datetime,
    *,
    to: Literal['minute', 'hour'] = 'minute',
) -> _dt.datetime:
    '''Round a datetime to the nearest minute or hour using half-up rounding.

    Parameters
    ----------
    dt : datetime.datetime
        The datetime to round.
    to : {'minute', 'hour'}, optional
        The unit to round to. Defaults to 'minute'.
    '''
    if to not in {'minute', 'hour'}:
        raise ValueError("'to' must be 'minute' or 'hour'")

    if to == 'minute':
        # Zero out seconds and microseconds, then adjust based on original seconds.
        base = dt.replace(second=0, microsecond=0)
        if dt.second >= 30:
            base += _dt.timedelta(minutes=1)
        return base

    # to == 'hour'
    base = dt.replace(minute=0, second=0, microsecond=0)
    if dt.minute >= 30:
        base += _dt.timedelta(hours=1)
    return base


if __name__ == '__main__':
    examples = [
        _dt.datetime(2023, 1, 1, 13, 29, 40),
        _dt.datetime(2023, 1, 1, 13, 29, 40),
        _dt.datetime(2023, 1, 1, 13, 31, 10),
    ]

    print('Round to minute:')
    print(examples[0], '->', round_datetime(examples[0], to='minute'))

    print('\nRound to hour:')
    print(examples[1], '->', round_datetime(examples[1], to='hour'))
    print(examples[2], '->', round_datetime(examples[2], to='hour'))


Round to minute:
2023-01-01 13:29:40 -> 2023-01-01 13:30:00

Round to hour:
2023-01-01 13:29:40 -> 2023-01-01 13:00:00
2023-01-01 13:31:10 -> 2023-01-01 14:00:00


---
### Exercise 4 – Converting Offset-Aware ISO Strings to Naive UTC `datetime`

Goal: Convert an ISO 8601 string that has an explicit UTC offset into a naive UTC `datetime`.

Scenario: You receive timestamps like `"2020-04-02T18:30:30-07:00"` that represent local time with an offset. For internal processing, you want everything as naive UTC.

Requirements:

1. Implement a function:
   ```python
   def to_utc_naive(iso_str: str) -> datetime.datetime:
       ...
   ```
2. Use:
   - `datetime.datetime.fromisoformat(iso_str)` to parse the string.
   - Validate that the result is aware (has a `tzinfo`). If it is naive, assume the string must include an offset and raise `ValueError` otherwise.
3. Convert the parsed datetime to UTC using `.astimezone(datetime.timezone.utc)`, then drop the `tzinfo` with `.replace(tzinfo=None)`.
4. Demonstrate with:
   - `"2020-04-02T18:30:30-07:00"`
   - `"2020-04-02T18:30:30+02:00"`
   - A string without offset, for example `"2020-04-02T18:30:30"`, to show the error handling.

This exercise practices:
- Parsing ISO 8601 datetimes
- Converting between time zones
- Normalizing to naive UTC datetimes for internal use


In [4]:
import datetime as _dt


def to_utc_naive(iso_str: str) -> _dt.datetime:
    '''Convert an offset-aware ISO 8601 string to a naive UTC datetime.

    Example input: '2020-04-02T18:30:30-07:00'.
    '''
    try:
        dt = _dt.datetime.fromisoformat(iso_str)
    except ValueError as exc:
        raise ValueError(f'Invalid ISO datetime string: {iso_str!r}') from exc

    if dt.tzinfo is None:
        raise ValueError(
            f'Datetime string must include an offset (tzinfo is naive): {iso_str!r}'
        )

    dt_utc = dt.astimezone(_dt.timezone.utc)
    return dt_utc.replace(tzinfo=None)


if __name__ == '__main__':
    examples = [
        '2020-04-02T18:30:30-07:00',
        '2020-04-02T18:30:30+02:00',
        '2020-04-02T18:30:30',  # no offset
    ]

    for s in examples:
        print(f'Converting {s!r}:')
        try:
            result = to_utc_naive(s)
            print('  -> UTC naive:', result, '(tzinfo:', result.tzinfo, ')')
        except ValueError as exc:
            print('  Error:', exc)
        print('-')


Converting '2020-04-02T18:30:30-07:00':
  -> UTC naive: 2020-04-03 01:30:30 (tzinfo: None )
-
Converting '2020-04-02T18:30:30+02:00':
  -> UTC naive: 2020-04-02 16:30:30 (tzinfo: None )
-
Converting '2020-04-02T18:30:30':
  Error: Datetime string must include an offset (tzinfo is naive): '2020-04-02T18:30:30'
-
